# Title

## 1. Imports, Configuration and Hyperparameters

In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
# to get current workind directory 
# Use os.getcwd()

import tensorflow as tf
print(tf.__version__)
from tensorflow import keras
from tensorflow.keras import layers, regularizers

# Use pandas to load dataset from csv file
import pandas as pd

# Configure GPU memory growth to be dynamic instead of allocating all memory at once
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# HYPERPARAMETERS
BATCH_SIZE = 64
WEIGHT_DECAY = 0.001
LEARNING_RATE = 0.001

2.4.1


## 2. Data Loading and Preprocessing

In [5]:
file_path = "/home/jay/Dev/datasets/multi_digit_mnist/"

train_df = pd.read_csv(file_path + "train.csv")
test_df = pd.read_csv(file_path + "test.csv")
train_images = file_path + "train_images/" + train_df.iloc[:, 0].values
test_images = file_path + "test_images/" + test_df.iloc[:, 0].values

train_labels = train_df.iloc[:, 1:].values
test_labels = test_df.iloc[:, 1:].values

@tf.autograph.experimental.do_not_convert
def read_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=1, dtype=tf.float32)

    # In older versions you need to set shape in order to avoid error
    # on newer (2.3.0+) the following 3 lines can safely be removed
    image.set_shape((64, 64, 1))
    label[0].set_shape([])
    label[1].set_shape([])

    labels = {"first_num": label[0], "second_num": label[1]}
    return image, labels

AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = (
    train_dataset.shuffle(buffer_size=len(train_labels))
    .map(read_image)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = (
    test_dataset.map(read_image)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

## 3. Model Definition

Sequential can only map from one input to one output, we use functional API to map from multiple inputs to multiple outputs. In this example we will map one input to two outptuts. As out multidigit_mnist dataset has 2 labels for each image.

In [6]:
# 64 x 64 pixels and 1 channel (grayscale)
inputs = keras.Input(shape=(64, 64, 1))
x = layers.Conv2D(
    filters=32,
    kernel_size=3,
    padding="same",
    kernel_regularizer=regularizers.l2(WEIGHT_DECAY),
)(inputs)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)

x = layers.Conv2D(
    64, 3, kernel_regularizer=regularizers.l2(WEIGHT_DECAY),
    )(x)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = layers.MaxPooling2D()(x)

x = layers.Conv2D(
    64, 3, activation="relu", kernel_regularizer=regularizers.l2(WEIGHT_DECAY),
)(x)

x = layers.Conv2D(128, 3, activation="relu")(x)
x = layers.MaxPooling2D()(x)

x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(64, activation="relu")(x)

# This is where function API differs from Sequential API, and is useful.
output1 = layers.Dense(10, activation="softmax", name="first_num")(x)
output2 = layers.Dense(10, activation="softmax", name="second_num")(x)

# Model with one input and two outputs
model = keras.Model(inputs=inputs, outputs=[output1, output2])

## 4. Compile Model

If you specify just one loss that will be assigned to all outputs. If you want to specify different losses for different outputs, you can do it by passing a dictionary individually.
In this example we have applied a softmax activation on the output layer hence we dont have to do logits=True.

In [7]:
model.compile(
    optimizer=keras.optimizers.Adam(LEARNING_RATE),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

## 5. Model Training and Evaluation

Batch size is taken care of in the dataloading part. We only need to specify the number of epochs.

In [9]:
print("Training model...")
model.fit(train_dataset, epochs=5, verbose=2)

print("Evaluating model...")
results = model.evaluate(test_dataset, verbose=0)
print(f"Test loss, test acc: {results[0]:.4f}")
print(f"First number loss: {results[1]:.4f}")
print(f"Second number loss: {results[2]:.4f}")
print(f"First number accuracy: {results[3]:.4f}")
print(f"Second number accuracy: {results[4]:.4f}")

Training model...
Epoch 1/5
1000/1000 - 25s - loss: 0.2085 - first_num_loss: 0.0849 - second_num_loss: 0.0815 - first_num_accuracy: 0.9731 - second_num_accuracy: 0.9727
Epoch 2/5
1000/1000 - 25s - loss: 0.1838 - first_num_loss: 0.0724 - second_num_loss: 0.0717 - first_num_accuracy: 0.9765 - second_num_accuracy: 0.9763
Epoch 3/5
1000/1000 - 25s - loss: 0.1724 - first_num_loss: 0.0690 - second_num_loss: 0.0649 - first_num_accuracy: 0.9777 - second_num_accuracy: 0.9785
Epoch 4/5
1000/1000 - 25s - loss: 0.1599 - first_num_loss: 0.0635 - second_num_loss: 0.0591 - first_num_accuracy: 0.9798 - second_num_accuracy: 0.9795
Epoch 5/5
1000/1000 - 25s - loss: 0.1454 - first_num_loss: 0.0559 - second_num_loss: 0.0544 - first_num_accuracy: 0.9822 - second_num_accuracy: 0.9818
Evaluating model...
Test loss, test acc: 1.0477
First number loss: 0.3249
Second number loss: 0.6882
First number accuracy: 0.9139
Second number accuracy: 0.8392
